In [93]:
from ypstruct import structure
import numpy as np

In [94]:
def run(problem,params):
    
    #  Problem info
    costfunc = problem.costfunc
    nvar = problem.nvar
    varmin = problem.varmin
    varmax = problem.varmax
    
    # paramiters
    maxit = params.maxit
    npop = params.npop
    pc = params.pc
    nc = int(np.round(pc*npop/2)*2)
    gamma = params.gamma
    mu = params.mu
    sigma = params.sigma
    
    # Empty Individual template
    empty_individual = structure()
    empty_individual.position = None
    empty_individual.cost = None
    
    # best solution found
    bestsol = empty_individual.deepcopy()
    bestsol.cost = np.inf
    
    #initialize population
    pop = empty_individual.repeat(npop)
    for i in range(0,npop):
        pop[i].position = np.random.uniform(varmin,varmax,nvar)
        pop[i].cost = costfunc(pop[i].position)
        if pop[i].cost<bestsol.cost:
            bestsol = pop[i].deepcopy()
        
    # best cost of iteration
    bestcost = np.empty(maxit)
    
    # main loop
    for it in range(maxit):
        
        popc = []
        for _ in range(nc//2):
            # select perents
            q = np.random.permutation(npop)
            p1 = pop[q[0]]
            p2 = pop[q[1]]
            
            # crossover
            c1, c2 =crossover(p1, p2, gamma)
            
            # mutetion
            c1 = mutate(c1, mu, sigma)
            c2 = mutate(c2, mu, sigma)
            
            # Apply Bounds
            apply_bound(c1, varmin, varmax)
            apply_bound(c2, varmin, varmax)
            
            #Evaluate first offspring
            c1.cost = costfunc(c1.position)
            if c1.cost < bestsol.cost:
                bestsol = c1.deepcopy()
            
            #Evaluate second offspring
            c2.cost = costfunc(c2.position)
            if c2.cost < bestsol.cost:
                bestsol = c2.deepcopy()
            
            # add offspring to population
            popc.append(c1)
            popc.append(c2)
            
        # merge, sort, and select
        pop += popc
        pop = sorted(pop, key=lambda x: x.cost)
        pop = pop[0:npop]
        
        #store best cost
        
        bestcost[it] = bestsol.cost
           
        # show iteration info
        print("Iteration {}: Best Cost = {}".format(it, bestcost[it]))

    # output
    out = structure()
    out.pop = pop
    out.bestsol = abs(bestsol)
    out.bestcost = abs(bestcost)
    return out

In [95]:
def crossover(p1, p2, gamma = 0.1):
    
    c1 = p1.deepcopy()
    c2 = p2.deepcopy()
    alpha = np.random.uniform(-gamma, 1+gamma, *c1.position.shape)
    c1.position = alpha*p1.position + (1-alpha)*p2.position
    c2.position = alpha*p2.position + (1-alpha)*p1.position
    return c1,c2

In [96]:
def mutate(x, mu, sigma):
    y = x.deepcopy()
    flag = np.random.rand(*x.position.shape) <= mu
    ind = np.argwhere(flag)
    y.position[ind] += sigma*np.random.randn(*ind.shape)
    return y

In [97]:
def apply_bound(x, varmin, varmax):
    x.position = np.maximum(x.position, varmin)
    x.position = np.minimum(x.position, varmax)